# Generacion de rutas

## Rutas Asignadas

El archivo de nodos en nuestro esquima de rutas esta compuesto por las rutas con detenciones registradas en el boletin de estadistica migratoria del INM y por los puntos de internacion terrestre mencionados en el mismo documento.

El df que contiene los puntos de internacion contempla solo a los de modalidad terrestre en la frontera Norte y Sur. Adicionalmente a los puntos que corresponden a la frontera Norte, los cuales en este ejercicio son los del final de la ruta, se les asigna un sector de la autoridad responsable en EE.UU. para la zona de cruce fronterizo correspondiente a cada punto.


In [2]:
import pandas as pd
from geopy.geocoders import Photon
from geopy.geocoders import Nominatim
from unidecode import unidecode
import unidecode
import numpy as np
from shapely.geometry import Point, Polygon
import geopandas as gpd

In [ ]:
ruta_archivo = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/1. Mapas/1. Mapas/Metodo_1/puntos_rutas/puntos de internacion/puntos_internacion.csv'

# Cargar el archivo .xls en un DataFrame
df_puntos_internacion = pd.read_csv(ruta_archivo)

In [ ]:
# Filtrar por valores 'FN' y 'FS' en la columna 'codigo_ruta'
df_fronteras = df_puntos_internacion[df_puntos_internacion['codigo_ruta'].isin(['FN', 'FS'])]


In [ ]:
# Cambiar el nombre de la columna 'edo' por 'estado'
df_fronteras = df_fronteras.rename(columns={'edo': 'estado'})

In [ ]:
# Agregar la columna 'detenciones' con todos los valores en 0
df_fronteras['detenciones'] = 0


In [ ]:
df_fronteras.columns.unique()

In [ ]:
ruta_archivo1 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/1. Mapas/1. Mapas/Metodo_1/puntos_rutas/ciudades detenciones/ciudades_detenciones_1.csv'

# Cargar el archivo .xls en un DataFrame
df_puntos_ciudades = pd.read_csv(ruta_archivo1)

In [ ]:
df_puntos_ciudades.columns.unique()

In [ ]:
# Agregar la columna 'nombre' con los valores de la columna 'ciudad'
df_puntos_ciudades['nombre'] = df_puntos_ciudades['ciudad']


In [ ]:
columnas_distintivas = ['nombre', 'ciudad', 'estado','detenciones', 'latitud', 'longitud', 'codigo_ruta']

# Seleccionar solo las columnas deseadas de cada DataFrame
df_puntos_ciudades_filtrado = df_puntos_ciudades[columnas_distintivas]
df_fronteras_filtrado = df_fronteras[columnas_distintivas]


In [ ]:
df_puntos_ciudades.head()

In [ ]:
# Verificar el cambio
print(df_puntos_ciudades[df_puntos_ciudades['ciudad'] == 'San Luis Potosi, San Luis Potosi'])

In [ ]:
# Localizar la fila donde la ciudad es 'San Luis Potosi' y actualizar latitud y longitud
df_puntos_ciudades.loc[df_puntos_ciudades['ciudad'] == 'San Luis Potosi, San Luis Potosi', 'latitud'] = 22.155654820940207
df_puntos_ciudades.loc[df_puntos_ciudades['ciudad'] == 'San Luis Potosi, San Luis Potosi', 'longitud'] = -100.97568521364161

# Verificar el cambio
print(df_puntos_ciudades[df_puntos_ciudades['ciudad'] == 'San Luis Potosi, San Luis Potosi'])


#### Concatenar los DataFrames

In [ ]:
# Concatenar las DataFrames filtradas
df_concatenada = pd.concat([df_puntos_ciudades_filtrado, df_fronteras_filtrado], ignore_index=True)



In [ ]:
df_concatenada['codigo_ruta'].unique()

In [ ]:
df_concatenada.head()

In [ ]:
# Especifica la ruta donde deseas guardar el archivo CSV
ruta_archivo_csv = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/1. Mapas/1. Mapas/Metodo_1/puntos_rutas/df_concatenada.csv'

# Guardar el DataFrame en el archivo CSV especificado
df_concatenada.to_csv(ruta_archivo_csv, index=False)

print(f"El DataFrame ha sido guardado exitosamente en: {ruta_archivo_csv}")


Resultado:

El archivo de nodos, contiene la informacion geodastica de las ubicaciones que corresponden a los puntos de recorrido dentro de una ruta. Estos de dividen de la siguiente manera.

la columna codigo_ruta, contiene las etiquetas de los siguientes puntos:

    FN: Corresponden a los puntos de internacion de la frontera norte, mas que un punto de corrido representan el final de las rutas y a traves de este punto se cruzan con los datos de los sectores USBP asignando un sector por su ubicacion en la frontera respecto a los puntos de internación FN.

    FS: Representan el inicio de los recorridos, por la naturaleza porosa de la frontera sur, los puntos que la componen estan asociados con las carreteras y zonas de recorrido principales, y directamente con las ciudades de paso.


    G = estados_ruta_golfo = ['Tabasco', 'Nuevo Leon', 'Veracruz', 'Tamaulipas']

    P = estados_ruta_pacifico = ['Guerrero', 'Michoacan', 'Jalisco', 'Nayarit', 'Sinaloa', 'Sonora', 'Baja California', 'Baja California Sur','Colima']

    C = estados_ruta_centro = ['San Luis Potosi','Estado de Mexico', 'Tlaxcala', 'Ciudad de Mexico', 'Puebla', 'Morelos', 'Hidalgo', 'Zacatecas', 'Durango', 'Chihuahua', 'Coahuila', 'Queretaro', 'Aguascalientes', 'Guanajuato']

    S = ruta_sur = ['Chiapas', 'Oaxaca', 'Tabasco', 'Campeche', 'Quintana Roo', 'Yucatan']
    
    

Nos sirven para dividir el pais en sectores y encontrar nuestras rutas principales por region, entendiendo la significancia en la eleccion de una region de transito sobre otra.

### Asignación etiqueta USBP a filas FN:

### Asignacion Etiqueta Grid y Codigo Unico

Se eligio el grid de cuadros de -7 por lado, ayudandonos a identificar las principales zonas de paso y una composición de combinaciones de rutas mas realista.

Al 

In [ ]:
ruta_archivo2 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/1. Mapas/1. Mapas/Metodo_1/puntos_rutas/df_concatenada.csv'

# Cargar el archivo .xls en un DataFrame
df_concatenada = pd.read_csv(ruta_archivo2)

In [ ]:
# Ruta al archivo Shapefile
ruta_shp = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/1. Mapas/2. arcgis(general)/grids/grid7x7/grid_punto7x7.shp'

# Leer el Shapefile en un GeoDataFrame
grid = gpd.read_file(ruta_shp)

# Verificar el contenido
print(grid.head())

In [ ]:
df_concatenada

In [ ]:
# Asegúrate de que 'df_concatenada' tenga las columnas 'latitud' y 'longitud'
df_concatenada['geometry'] = [Point(x, y) for x, y in zip(df_concatenada['longitud'], df_concatenada['latitud'])]
gdf_concatenada = gpd.GeoDataFrame(df_concatenada, geometry='geometry')


In [ ]:
# Asumiendo que 'grid' es tu GeoDataFrame de polígonos y ya está cargado y tiene un CRS definido
gdf_concatenada.crs = grid.crs


In [ ]:
# Realiza la unión espacial usando sjoin
point_in_poly = gpd.sjoin(gdf_concatenada, grid, how='left', op='within')


In [ ]:
# Verificar si la columna 'ID_Poligono' está presente después del sjoin
print(point_in_poly.columns)


In [ ]:
# Ordenar por 'ID_Poligono' y 'longitud'
point_in_poly.sort_values(by=['ID_Poligon', 'longitud'], inplace=True)

# Crear la columna 'secuencia'
point_in_poly['secuencia'] = point_in_poly.groupby('ID_Poligon').cumcount() + 1

# Seleccionar las columnas relevantes, si 'ID_Poligono' existe
df_concatenada_final = point_in_poly[['nombre', 'ciudad', 'estado','detenciones', 'latitud', 'longitud', 'codigo_ruta', 'ID_Poligon', 'secuencia']]


In [ ]:
# Asegúrate de que el DataFrame 'point_in_poly' tiene las columnas 'codigo_ruta', 'ID_Poligono' y 'secuencia'

# Generar la columna 'codigo_unico' mediante la concatenación de las columnas
df_concatenada_final['codigo_unico'] = df_concatenada_final['codigo_ruta'] + "-" + df_concatenada_final['ID_Poligon'] + "-" + df_concatenada_final['secuencia'].astype(str)

# Verificar el resultado
print(df_concatenada_final[['codigo_ruta', 'ID_Poligon', 'secuencia', 'codigo_unico']].head())


In [ ]:
df_concatenada_final

### PS. df_concatenada_final.geometry_reach
Un shape de 10km de radio al rededor del geometryxy de cada punto)

In [ ]:
# Suponiendo que df_concatenada_final es tu DataFrame y tiene las columnas especificadas
df_concatenada_final_georeach = df_concatenada_final[['nombre', 'ciudad', 'estado', 'detenciones', 'latitud', 'longitud', 'codigo_ruta', 'ID_Poligon']]

# Verificar el resultado imprimiendo las primeras filas del DataFrame resultante
print(df_concatenada_final_georeach.head())


In [ ]:
# Asegurándonos de trabajar con el DataFrame correcto, df1
gdf = gpd.GeoDataFrame(df_concatenada_final_georeach, geometry=gpd.points_from_xy(df_concatenada_final_georeach.longitud, df_concatenada_final_georeach.latitud))

# Estableciendo el CRS original a EPSG:4326 para las coordenadas geográficas
gdf.crs = "EPSG:4326"

# Cambiando el CRS a uno que use metros para el cálculo del buffer. Reemplazando el EPSG con un código correcto.
# Nota: El EPSG 6366 parece ser un error. Asegúrate de usar el código EPSG de UTM adecuado para tu zona.
gdf_metros = gdf.to_crs(epsg=6366)  # Reemplaza YOUR_UTM_EPSG_CODE con el código EPSG adecuado

# Aplicando el buffer de 10km para crear una nueva columna 'geometry_range' con los polígonos
gdf_metros['geometry_range'] = gdf_metros.geometry.buffer(30000)

# (Opcional) Si deseas convertir el GeoDataFrame de regreso a lat/long (EPSG:4326) para alguna otra operación,
# y quieres que la columna 'geometry_range' sea la geometría activa,
# debes hacerlo antes de cambiar el CRS de regreso a EPSG:4326.
# gdf_metros.set_geometry('geometry_range', inplace=True)

# Cambiando de regreso a EPSG:4326
gdf_final = gdf_metros.to_crs(epsg=4326)

In [ ]:
# Eliminar la columna 'geometry_range' si no es necesaria
if 'geometry' in gdf_final.columns:
    gdf_final.drop(columns=['geometry'], inplace=True)

# Asegúrate de que 'geometry' es la columna geométrica correcta
gdf_final.set_geometry('geometry_range', inplace=True)

In [ ]:
gdf_final.head()

In [ ]:
# Guardar grid_1 como Shapefile
gdf_final.to_file("/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/1. Mapas/2. arcgis(general)/grids/gridrange/gridrange.shp")


## Asignamos pesos

df_valores = df_concatenada_final

geometry_reach = gdf_final

to

df_valores[]


In [ ]:
# Asegúrate de que todas las columnas listadas estén en df_concatenada_final y de que los nombres de las columnas estén escritos correctamente.
df_valores = df_concatenada_final[['nombre', 'ciudad', 'estado', 'latitud', 'longitud', 'codigo_ruta', 'ID_Poligon', 'codigo_unico', 'detenciones']]


In [ ]:
df_valores.head()

### 1. Detenciones INM (Ya viene en el df inicial por que corresponde a esa df)

### 2. Instituciones INM (Aproximacion Radial: 30km)
Hay que cargar nuestra bd con ubicaciones del INM

In [ ]:
geometry_reach = gdf_final
geometry_reach.head()

In [5]:
ruta_archivo4 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/1. Mapas/1. Mapas/Metodo_1/estaciones INM/inm_gdp.csv'

# Cargar el archivo .xls en un DataFrame
inm = pd.read_csv(ruta_archivo4)

In [6]:
inm.head(60)

,name,tipo,sub_tipo,latitud,longitud,ciudad,estado,ID_Poligono
0,Acayucan Estacion Migratoria,INM,Estacion Migratoria INM,18.001851,-94.921076,Acayucan,Veracruz,PG1378
1,Agua Prieta Estacion Provisional,INM,Estacion Migratoria INM,31.332947,-109.560374,Agua Prieta,Sonora,PG578
2,Aguascalientes Estacion Migratoria,INM,Estacion Migratoria INM,21.929551,-102.320223,Aguascalientes,Aguascalientes,PG994
3,Campeche Estacion Provisional,INM,Estacion Migratoria INM,19.845660,-90.539741,Campeche,Campeche,PG1654
4,Centro de Atencion Integral al Trafico Fronter...,INM,Centro de Atencion Integral,16.149934,-92.072111,La Trinitaria,Chiapas,PG1531
5,Centro de Atencion Integral al Transito Fronte...,INM,Centro de Atencion Integral,15.162749,-92.532987,Salto de Agua,Chiapas,PG1530
6,Chetumal Estacion Migratoria,INM,Estacion Migratoria INM,18.526295,-88.283332,Chetumal,Quintana Roo,PG1769
7,Chihuahua Estacion Migratoria,INM,Estacion Migratoria INM,28.642257,-106.084403,Chihuahua,Chihuahua,PG769
8,Ciudad Cuauhtemoc Estacion Provisional,INM,Estacion Migratoria INM,15.667668,-92.002344,Ciudad Cuauhtemoc,Chiapas,PG1531
9,Ciudad del Carmen Estacion Provisional,INM,Estacion Migratoria INM,18.642626,-91.823607,Ciudad del Carmen,Campeche,PG1574


In [ ]:
# Convertir 'inm' a un GeoDataFrame
gdf_inm = gpd.GeoDataFrame(
    inm, 
    geometry=gpd.points_from_xy(inm.longitud, inm.latitud),
    crs="EPSG:4326"  # Asegúrate de usar el sistema de coordenadas correcto aquí
)

# Asegurar que ambos GeoDataFrames tienen el mismo sistema de coordenadas
gdf_inm = gdf_inm.to_crs(geometry_reach.crs)

# Ahora, realiza la operación espacial
# Contar cuántos puntos caen dentro de cada polígono en geometry_reach
conteos = gpd.sjoin(gdf_inm, geometry_reach, predicate='within').groupby('index_right').size()

# 'index_right' es el índice de los polígonos en 'geometry_reach' donde caen los puntos
# Agrega el conteo al 'geometry_reach' DataFrame
geometry_reach['conteo_puntos'] = geometry_reach.index.map(conteos).fillna(0).astype(int)

# Verificar los resultados
print(geometry_reach[['geometry_range', 'conteo_puntos']])

In [ ]:
# Realizar un merge (join) entre 'df_valores' y 'geometry_reach' utilizando la columna 'ciudad' como clave
df_valores = df_valores.merge(geometry_reach[['ciudad', 'conteo_puntos']],
                              on='ciudad',
                              how='left')




In [ ]:
# Cambiar el nombre de la columna 'conteo_puntos' a 'inm_30km'
df_valores = df_valores.rename(columns={'conteo_puntos': 'inm_30km'})



In [ ]:
# Cambiar el nombre de la columna 'ID_Poligon' a 'ID_Poligono'
df_valores.rename(columns={'ID_Poligon': 'ID_Poligono'}, inplace=True)

# Verificar el


In [ ]:
df_valores1.tail(60)

In [48]:
# Especifica la ruta donde deseas guardar el archivo CSV
ruta_archivo_csv1 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/1. Mapas/1. Mapas/Metodo_1/df_valores_rutas.csv'

# Guardar el DataFrame en el archivo CSV especificado
#df_valores1.to_csv(ruta_archivo_csv1, index=False)

#print(f"El DataFrame ha sido guardado exitosamente en: {ruta_archivo_csv1}")

#### 2.1Instituciones INM (En el mismo grid)


In [36]:
df_valores1 = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/1. Mapas/1. Mapas/Metodo_1/df_valores_rutas.csv')

In [37]:
df_valores1.head()

,nombre,ciudad,estado,latitud,longitud,codigo_ruta,ID_Poligono,codigo_unico,detenciones,inm_30km
0,"Zihuatanejo de Azueta, Guerrero","Zihuatanejo de Azueta, Guerrero",Guerrero,17.807900,-101.496952,P,PG1027,P-PG1027-1,14,1
1,"Morelia, Michoacan","Morelia, Michoacan",Michoacan,19.702712,-101.192382,P,PG1029,P-PG1029-1,137,0
2,"Cuitzeo, Michoacan","Cuitzeo, Michoacan",Michoacan,19.969314,-101.141220,P,PG1030,P-PG1030-1,1,0
3,"Leon, Guanajuato","Leon, Guanajuato",Guanajuato,21.121860,-101.682508,C,PG1031,C-PG1031-1,2,0
4,"Irapuato, Guanajuato","Irapuato, Guanajuato",Guanajuato,20.675876,-101.352105,C,PG1031,C-PG1031-2,44,0


In [38]:
# Utiliza value_counts() para obtener el conteo de cada ID_Poligono único
conteo_id_poligono = inm['ID_Poligono'].value_counts().reset_index()
conteo_id_poligono.columns = ['ID_Poligono', 'conteo']



In [39]:
# Realizar un merge (join) entre 'df_valores1' y 'conteo_id_poligono' utilizando la columna 'ID_Poligono' como clave
df_valores1 = df_valores1.merge(conteo_id_poligono[['ID_Poligono', 'conteo']],
                                on='ID_Poligono',
                                how='left')

# Reemplazar NaN por 0 en la columna 'conteo'
df_valores1['conteo'] = df_valores1['conteo'].fillna(0).astype(int)

# Asegurarse de que todos los valores sean un solo dígito
df_valores1['conteo'] = df_valores1['conteo'].apply(lambda x: min(x, 9))




In [40]:
# Cambiar el nombre de la columna 'conteo' a 'inm_in_grid'
df_valores1 = df_valores1.rename(columns={'conteo': 'inm_in_grid'})


In [41]:
print(df_valores1.sort_values(by='inm_in_grid', ascending=False).head())


                         nombre                      ciudad   estado  \
305          Escuintla, Chiapas          Escuintla, Chiapas  Chiapas   
310          Tapachula, Chiapas          Tapachula, Chiapas  Chiapas   
304  Villa Comaltitlan, Chiapas  Villa Comaltitlan, Chiapas  Chiapas   
318      La Trinitaria, Chiapas      La Trinitaria, Chiapas  Chiapas   
317  Frontera Comalapa, Chiapas  Frontera Comalapa, Chiapas  Chiapas   

       latitud   longitud codigo_ruta ID_Poligono codigo_unico detenciones  \
305  15.351003 -92.555765           S      PG1530   S-PG1530-3          19   
310  14.904291 -92.260926           S      PG1530   S-PG1530-8      98 486   
304  15.170973 -92.612886           S      PG1530   S-PG1530-2       1 835   
318  16.120616 -92.054243           S      PG1531   S-PG1531-4         177   
317  15.659014 -92.142222           S      PG1531   S-PG1531-3         802   

     inm_30km  inm_in_grid  
305         2            4  
310         2            4  
304        

#### 2.2 Instituciones INM (En el mismo estado)

In [42]:
# Utiliza value_counts() para obtener el conteo de cada ID_Poligono único
conteo_id_poligono1 = inm['estado'].value_counts().reset_index()
conteo_id_poligono1.columns = ['estado', 'conteo_edo']


In [43]:
conteo_id_poligono1.head()

,estado,conteo_edo
0,Chiapas,13
1,Coahuila,4
2,Oaxaca,4
3,Veracruz,3
4,Campeche,3


In [44]:
# Realizar un merge (join) entre 'df_valores1' y 'conteo_id_poligono' utilizando la columna 'ID_Poligono' como clave
df_valores1 = df_valores1.merge(conteo_id_poligono1[['estado', 'conteo_edo']],
                                on='estado',
                                how='left')

# Reemplazar NaN por 0 en la columna 'conteo'
df_valores1['conteo_edo'] = df_valores1['conteo_edo'].fillna(0).astype(int)



In [45]:
print(df_valores1.sort_values(by='conteo_edo', ascending=False).head())

                    nombre                 ciudad   estado    latitud  \
301  Tuxtla Chico, Chiapas  Tuxtla Chico, Chiapas  Chiapas  14.876862   
307     Huehuetan, Chiapas     Huehuetan, Chiapas  Chiapas  15.021716   
309      Siltepec, Chiapas      Siltepec, Chiapas  Chiapas  15.556149   
310     Tapachula, Chiapas     Tapachula, Chiapas  Chiapas  14.904291   
311   Talismán (terrestre)     Talismán , Chiapas  Chiapas  14.965561   

      longitud codigo_ruta ID_Poligono codigo_unico detenciones  inm_30km  \
301 -92.196752           S      PG1529   S-PG1529-3         595         2   
307 -92.428670           S      PG1530   S-PG1530-5      18 319         4   
309 -92.322603           S      PG1530   S-PG1530-7           1         0   
310 -92.260926           S      PG1530   S-PG1530-8      98 486         2   
311 -92.146625          FS      PG1530  FS-PG1530-9           0         2   

     inm_in_grid  conteo_edo  
301            0          13  
307            4          13  
309  

In [46]:
df_valores1.tail(60)

,nombre,ciudad,estado,latitud,longitud,codigo_ruta,ID_Poligono,codigo_unico,detenciones,inm_30km,inm_in_grid,conteo_edo
370,"San Luis Rio Colorado, Sonora","San Luis Rio Colorado, Sonora",Sonora,32.451796,-114.765254,P,PG307,P-PG307-1,67,0,0,2
371,San Luis Río Colorado (terrestre),"San Luis Río Colorado , Sonora",Sonora,32.472353,-114.742716,FN,PG307,FN-PG307-2,0,0,0,2
372,Algodones (terrestre),"Algodones , Baja California",Baja California,32.714894,-114.733311,FN,PG307,FN-PG307-3,0,0,0,2
373,"Puerto Penasco, Sonora","Puerto Penasco, Sonora",Sonora,31.317087,-113.531819,P,PG383,P-PG383-1,1,0,0,2
374,"General Plutarco Elias Calles, Sonora","General Plutarco Elias Calles, Sonora",Sonora,31.694984,-112.711749,P,PG422,P-PG422-1,1 431,0,0,2
375,Sonoyta (terrestre) 14,"Sonoyta , Sonora",Sonora,31.879225,-112.818481,FN,PG423,FN-PG423-1,0,0,0,2
376,"Caborca, Sonora","Caborca, Sonora",Sonora,30.716601,-112.156260,P,PG460,P-PG460-1,414,0,0,2
377,"Altar, Sonora","Altar, Sonora",Sonora,30.717032,-111.838004,P,PG460,P-PG460-2,30,0,0,2
378,"Hermosillo, Sonora","Hermosillo, Sonora",Sonora,29.094821,-110.969220,P,PG497,P-PG497-1,9 637,1,1,2
379,"Santa Ana, Sonora","Santa Ana, Sonora",Sonora,30.541092,-111.118488,P,PG499,P-PG499-1,12 341,0,0,2


## Asignacion Instituciones SC

In [54]:
sc = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/1. Mapas/1. Mapas/Metodo_1/SC/instituciones_sc_1.csv')

### aprox 30km

In [53]:
georeach = gpd.read_file('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/1. Mapas/2. arcgis(general)/grids/gridrange/gridrange.shp')

In [49]:
df_valores2 = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/1. Mapas/1. Mapas/Metodo_1/df_valores_rutas.csv')

In [57]:
# Convertir 'inm' a un GeoDataFrame
gdf_inm = gpd.GeoDataFrame(
    sc, 
    geometry=gpd.points_from_xy(sc.longitud, sc.latitud),
    crs="EPSG:4326"  # Asegúrate de usar el sistema de coordenadas correcto aquí
)

# Asegurar que ambos GeoDataFrames tienen el mismo sistema de coordenadas
gdf_inm = gdf_inm.to_crs(georeach.crs)

# Ahora, realiza la operación espacial
# Contar cuántos puntos caen dentro de cada polígono en geometry_reach
conteos = gpd.sjoin(gdf_inm, georeach, predicate='within').groupby('index_right').size()

# 'index_right' es el índice de los polígonos en 'geometry_reach' donde caen los puntos
# Agrega el conteo al 'geometry_reach' DataFrame
georeach['conteo_puntos'] = georeach.index.map(conteos).fillna(0).astype(int)

# Verificar los resultados
print(georeach[['geometry', 'conteo_puntos']])

                                              geometry  conteo_puntos
0    POLYGON ((2189564.100 2038820.103, 2189419.641...              0
1    POLYGON ((2203164.724 2257818.750, 2203020.266...              0
2    POLYGON ((2205757.540 2288739.937, 2205613.081...              0
3    POLYGON ((2134877.081 2414244.870, 2134732.623...              2
4    POLYGON ((2175208.825 2367005.198, 2175064.367...              5
..                                                 ...            ...
419  POLYGON ((2113407.075 2216493.661, 2113262.617...              0
420  POLYGON ((2042405.596 2498304.969, 2042261.138...             26
421  POLYGON ((2054061.510 2549327.296, 2053917.051...              1
422  POLYGON ((2061749.918 2516224.034, 2061605.460...             25
423  POLYGON ((2044147.752 2590077.663, 2044003.294...              2

[424 rows x 2 columns]


In [59]:
# Realizar un merge (join) entre 'df_valores' y 'geometry_reach' utilizando la columna 'ciudad' como clave
df_valores2 = df_valores2.merge(georeach[['ciudad', 'conteo_puntos']],
                              on='ciudad',
                              how='left')

In [70]:
df_valores2

,nombre,ciudad,estado,latitud,longitud,codigo_ruta,ID_Poligono,codigo_unico,detenciones,inm_30km,inm_in_grid,conteo_edo,conteo_puntos,sc_in_grid,sc_in_estado
0,"Zihuatanejo de Azueta, Guerrero","Zihuatanejo de Azueta, Guerrero",Guerrero,17.807900,-101.496952,P,PG1027,P-PG1027-1,14,1,0,1,0,0,1
1,"Morelia, Michoacan","Morelia, Michoacan",Michoacan,19.702712,-101.192382,P,PG1029,P-PG1029-1,137,0,0,0,0,0,1
2,"Cuitzeo, Michoacan","Cuitzeo, Michoacan",Michoacan,19.969314,-101.141220,P,PG1030,P-PG1030-1,1,0,0,0,0,1,1
3,"Leon, Guanajuato","Leon, Guanajuato",Guanajuato,21.121860,-101.682508,C,PG1031,C-PG1031-1,2,0,0,0,2,8,23
4,"Irapuato, Guanajuato","Irapuato, Guanajuato",Guanajuato,20.675876,-101.352105,C,PG1031,C-PG1031-2,44,0,0,0,5,8,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,"Uruapan, Michoacan","Uruapan, Michoacan",Michoacan,19.414727,-102.052265,P,PG990,P-PG990-1,1,0,0,0,0,0,1
438,"Jesus Maria, Aguascalientes","Jesus Maria, Aguascalientes",Aguascalientes,21.934555,-102.468452,C,PG994,C-PG994-1,20,1,1,2,26,11,29
439,"Cosio, Aguascalientes","Cosio, Aguascalientes",Aguascalientes,22.370515,-102.312009,C,PG994,C-PG994-2,98,0,1,2,1,11,29
440,"San Francisco de los Romo, Aguascalientes","San Francisco de los Romo, Aguascalientes",Aguascalientes,22.074628,-102.270713,C,PG994,C-PG994-3,18,1,1,2,25,11,29


### sc in same grid

In [64]:
# Utiliza value_counts() para obtener el conteo de cada ID_Poligono único
conteo_id_poligono1 = sc['ID_Poligono'].value_counts().reset_index()
conteo_id_poligono1.columns = ['ID_Poligono', 'sc_in_grid']


In [65]:
# Realizar un merge (join) entre 'df_valores1' y 'conteo_id_poligono' utilizando la columna 'ID_Poligono' como clave
df_valores2 = df_valores2.merge(conteo_id_poligono1[['ID_Poligono', 'sc_in_grid']],
                                on='ID_Poligono',
                                how='left')

# Reemplazar NaN por 0 en la columna 'conteo'
df_valores2['sc_in_grid'] = df_valores2['sc_in_grid'].fillna(0).astype(int)


### sc in estado

In [68]:
# Utiliza value_counts() para obtener el conteo de cada ID_Poligono único
conteo_id_poligono2 = sc['estado'].value_counts().reset_index()
conteo_id_poligono2.columns = ['estado', 'sc_in_estado']

In [69]:
# Realizar un merge (join) entre 'df_valores1' y 'conteo_id_poligono' utilizando la columna 'ID_Poligono' como clave
df_valores2 = df_valores2.merge(conteo_id_poligono2[['estado', 'sc_in_estado']],
                                on='estado',
                                how='left')

# Reemplazar NaN por 0 en la columna 'conteo'
df_valores2['sc_in_estado'] = df_valores2['sc_in_estado'].fillna(0).astype(int)

In [71]:
conteo_id_poligono2

,estado,sc_in_estado
0,Ciudad de Mexico,67
1,Baja California,59
2,Chiapas,35
3,Aguascalientes,29
4,Coahuila,27
5,Sonora,23
6,Guanajuato,23
7,Tamaulipas,16
8,Veracruz,12
9,Jalisco,11


In [72]:
print(df_valores2.sort_values(by='sc_in_estado', ascending=False).head())

                                    nombre  \
72   Venustiano Carranza, Ciudad de Mexico   
70              Coyoacan, Ciudad de Mexico   
71     Gustavo A. Madero, Ciudad de Mexico   
73            Iztapalapa, Ciudad de Mexico   
373               Tijuana, Baja California   

                                    ciudad            estado    latitud  \
72   Venustiano Carranza, Ciudad de Mexico  Ciudad de Mexico  19.419279   
70              Coyoacan, Ciudad de Mexico  Ciudad de Mexico  19.349885   
71     Gustavo A. Madero, Ciudad de Mexico  Ciudad de Mexico  19.483169   
73            Iztapalapa, Ciudad de Mexico  Ciudad de Mexico  19.359119   
373               Tijuana, Baja California   Baja California  32.531740   

       longitud codigo_ruta ID_Poligono codigo_unico detenciones  inm_30km  \
72   -99.113564           C      PG1146  C-PG1146-10       1 851         1   
70   -99.162360           C      PG1146   C-PG1146-8           7         1   
71   -99.113789           C      PG114

In [75]:
# Cambiar los nombres de las columnas 'conteo_edo' y 'conteo_puntos'
df_valores2 = df_valores2.rename(columns={'conteo_edo': 'inm_in_estado',
                                          'conteo_puntos': 'sc_30km'})

# Verificar los cambios
print(df_valores2.head())


                            nombre                           ciudad  \
0  Zihuatanejo de Azueta, Guerrero  Zihuatanejo de Azueta, Guerrero   
1               Morelia, Michoacan               Morelia, Michoacan   
2               Cuitzeo, Michoacan               Cuitzeo, Michoacan   
3                 Leon, Guanajuato                 Leon, Guanajuato   
4             Irapuato, Guanajuato             Irapuato, Guanajuato   

       estado    latitud    longitud codigo_ruta ID_Poligono codigo_unico  \
0    Guerrero  17.807900 -101.496952           P      PG1027   P-PG1027-1   
1   Michoacan  19.702712 -101.192382           P      PG1029   P-PG1029-1   
2   Michoacan  19.969314 -101.141220           P      PG1030   P-PG1030-1   
3  Guanajuato  21.121860 -101.682508           C      PG1031   C-PG1031-1   
4  Guanajuato  20.675876 -101.352105           C      PG1031   C-PG1031-2   

  detenciones  inm_30km  inm_in_grid  inm_in_estado  sc_30km  sc_in_grid  \
0          14         1           

In [76]:
# Especifica la ruta donde deseas guardar el archivo CSV
ruta_archivo_csv2 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/1. Mapas/1. Mapas/Metodo_1/df_valores_rutas.csv'

# Guardar el DataFrame en el archivo CSV especificado
df_valores2.to_csv(ruta_archivo_csv2, index=False)

In [77]:
dfvalores3 = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/1. Mapas/1. Mapas/Metodo_1/df_valores_rutas.csv')

In [78]:
dfvalores3

,nombre,ciudad,estado,latitud,longitud,codigo_ruta,ID_Poligono,codigo_unico,detenciones,inm_30km,inm_in_grid,inm_in_estado,sc_30km,sc_in_grid,sc_in_estado
0,"Zihuatanejo de Azueta, Guerrero","Zihuatanejo de Azueta, Guerrero",Guerrero,17.807900,-101.496952,P,PG1027,P-PG1027-1,14,1,0,1,0,0,1
1,"Morelia, Michoacan","Morelia, Michoacan",Michoacan,19.702712,-101.192382,P,PG1029,P-PG1029-1,137,0,0,0,0,0,1
2,"Cuitzeo, Michoacan","Cuitzeo, Michoacan",Michoacan,19.969314,-101.141220,P,PG1030,P-PG1030-1,1,0,0,0,0,1,1
3,"Leon, Guanajuato","Leon, Guanajuato",Guanajuato,21.121860,-101.682508,C,PG1031,C-PG1031-1,2,0,0,0,2,8,23
4,"Irapuato, Guanajuato","Irapuato, Guanajuato",Guanajuato,20.675876,-101.352105,C,PG1031,C-PG1031-2,44,0,0,0,5,8,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,"Uruapan, Michoacan","Uruapan, Michoacan",Michoacan,19.414727,-102.052265,P,PG990,P-PG990-1,1,0,0,0,0,0,1
438,"Jesus Maria, Aguascalientes","Jesus Maria, Aguascalientes",Aguascalientes,21.934555,-102.468452,C,PG994,C-PG994-1,20,1,1,2,26,11,29
439,"Cosio, Aguascalientes","Cosio, Aguascalientes",Aguascalientes,22.370515,-102.312009,C,PG994,C-PG994-2,98,0,1,2,1,11,29
440,"San Francisco de los Romo, Aguascalientes","San Francisco de los Romo, Aguascalientes",Aguascalientes,22.074628,-102.270713,C,PG994,C-PG994-3,18,1,1,2,25,11,29


In [79]:
# Lista de columnas para convertir a numérico
columnas_a_convertir = ['detenciones', 'inm_30km', 'inm_in_grid', 'inm_in_estado', 'sc_30km', 'sc_in_grid', 'sc_in_estado']

# Convertir todas las columnas especificadas a numérico
for columna in columnas_a_convertir:
    dfvalores3[columna] = pd.to_numeric(dfvalores3[columna], errors='coerce')

# Reemplazar NaN por 0 si es necesario
dfvalores3[columnas_a_convertir] = dfvalores3[columnas_a_convertir].fillna(0)

# Convertir a enteros si no hay decimales
dfvalores3[columnas_a_convertir] = dfvalores3[columnas_a_convertir].astype(int)

# Verificar los cambios
print(dfvalores3[columnas_a_convertir].head())


   detenciones  inm_30km  inm_in_grid  inm_in_estado  sc_30km  sc_in_grid  \
0           14         1            0              1        0           0   
1          137         0            0              0        0           0   
2            1         0            0              0        0           1   
3            2         0            0              0        2           8   
4           44         0            0              0        5           8   

   sc_in_estado  
0             1  
1             1  
2             1  
3            23  
4            23  


In [80]:
dfvalores3

,nombre,ciudad,estado,latitud,longitud,codigo_ruta,ID_Poligono,codigo_unico,detenciones,inm_30km,inm_in_grid,inm_in_estado,sc_30km,sc_in_grid,sc_in_estado
0,"Zihuatanejo de Azueta, Guerrero","Zihuatanejo de Azueta, Guerrero",Guerrero,17.807900,-101.496952,P,PG1027,P-PG1027-1,14,1,0,1,0,0,1
1,"Morelia, Michoacan","Morelia, Michoacan",Michoacan,19.702712,-101.192382,P,PG1029,P-PG1029-1,137,0,0,0,0,0,1
2,"Cuitzeo, Michoacan","Cuitzeo, Michoacan",Michoacan,19.969314,-101.141220,P,PG1030,P-PG1030-1,1,0,0,0,0,1,1
3,"Leon, Guanajuato","Leon, Guanajuato",Guanajuato,21.121860,-101.682508,C,PG1031,C-PG1031-1,2,0,0,0,2,8,23
4,"Irapuato, Guanajuato","Irapuato, Guanajuato",Guanajuato,20.675876,-101.352105,C,PG1031,C-PG1031-2,44,0,0,0,5,8,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,"Uruapan, Michoacan","Uruapan, Michoacan",Michoacan,19.414727,-102.052265,P,PG990,P-PG990-1,1,0,0,0,0,0,1
438,"Jesus Maria, Aguascalientes","Jesus Maria, Aguascalientes",Aguascalientes,21.934555,-102.468452,C,PG994,C-PG994-1,20,1,1,2,26,11,29
439,"Cosio, Aguascalientes","Cosio, Aguascalientes",Aguascalientes,22.370515,-102.312009,C,PG994,C-PG994-2,98,0,1,2,1,11,29
440,"San Francisco de los Romo, Aguascalientes","San Francisco de los Romo, Aguascalientes",Aguascalientes,22.074628,-102.270713,C,PG994,C-PG994-3,18,1,1,2,25,11,29


In [81]:
# Especifica la ruta donde deseas guardar el archivo CSV
ruta_archivo_csv2 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/1. Mapas/1. Mapas/Metodo_1/df_valores_rutas.csv'

# Guardar el DataFrame en el archivo CSV especificado
dfvalores3.to_csv(ruta_archivo_csv2, index=False)